In [ ]:
#ラベリングによる学習/検証データの準備

from PIL import Image
import os, glob
import numpy as np
import random, math

#画像が保存されているルートディレクトリのパス
root_dir = "./images"
# 商品名
categories = ["醤油ラーメン","味噌ラーメン","塩ラーメン","豚骨ラーメン","担々麺"]

# 画像データ用配列
X = []
# ラベルデータ用配列
Y = []

#画像データごとにadd_sample()を呼び出し、X,Yの配列を返す関数
def make_sample(files):
    global X, Y
    X = []
    Y = []
    for cat, fname in files:
        add_sample(cat, fname)
    return np.array(X), np.array(Y)

#渡された画像データを読み込んでXに格納し、また、
#画像データに対応するcategoriesのidxをY格納する関数
def add_sample(cat, fname):
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((256, 256))
    data = np.asarray(img)
    X.append(data)
    Y.append(cat)

#全データ格納用配列
allfiles = []

#カテゴリ配列の各値と、それに対応するidxを認識し、全データをallfilesにまとめる
for idx, cat in enumerate(categories):
    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir + "/*.jpg")
    for f in files:
        allfiles.append((idx, f))

#シャッフル後、学習データと検証データに分ける
random.shuffle(allfiles)
th = math.floor(len(allfiles) * 0.8)
train = allfiles[0:th]
test  = allfiles[th:]
X_train, y_train = make_sample(train)
X_test, y_test = make_sample(test)
xy = (X_train, X_test, y_train, y_test)
#データを保存する（データの名前を「tea_data.npy」としている）
np.save("./data/ramen_data.npy", xy)

In [ ]:
# テストデータを作成
from PIL import Image
import os, glob
import numpy as np
import random, math

# 画像が保存されているディレクトリのパス
root_dir = "./images"
# 画像が保存されているフォルダ名
categories = ["醤油ラーメン","味噌ラーメン","塩ラーメン","豚骨ラーメン","担々麺"]

X = [] # 画像データ
Y = [] # ラベルデータ

# フォルダごとに分けられたファイルを収集
#（categoriesのidxと、画像のファイルパスが紐づいたリストを生成）
allfiles = []
for idx, cat in enumerate(categories):
    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir + "/*.jpg")
    for f in files:
        allfiles.append((idx, f))

for cat, fname in allfiles:
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((256, 256))
    data = np.asarray(img)
    X.append(data)
    Y.append(cat)

x = np.array(X)
y = np.array(Y)

np.save("./data/ramen_data_test_X_150.npy", x)
np.save("./data/ramen_data_test_Y_150.npy", y)

In [1]:
#モデルの構築

from keras import layers, models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation="relu"))
model.add(layers.Dense(5,activation="sigmoid")) #分類先の種類分設定

#モデル構成の確認
model.summary()

Using TensorFlow backend.
W0819 15:44:41.693379 4373079488 deprecation_wrapper.py:119] From /Users/sakurai/Documents/learn-tensorflow/tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0819 15:44:41.703962 4373079488 deprecation_wrapper.py:119] From /Users/sakurai/Documents/learn-tensorflow/tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 15:44:41.705894 4373079488 deprecation_wrapper.py:119] From /Users/sakurai/Documents/learn-tensorflow/tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0819 15:44:41.716428 4373079488 deprecation_wrapper.py:119] From /Users/sakurai/Documents/learn-tensorflow/tensorflow/lib/python3.7/site-packages/

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 128)       147584    
__________

In [2]:
#モデルのコンパイル

from keras import optimizers

model.compile(loss="binary_crossentropy",
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=["acc"])

W0819 15:44:41.787335 4373079488 deprecation_wrapper.py:119] From /Users/sakurai/Documents/learn-tensorflow/tensorflow/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0819 15:44:41.792314 4373079488 deprecation_wrapper.py:119] From /Users/sakurai/Documents/learn-tensorflow/tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0819 15:44:41.795454 4373079488 deprecation.py:323] From /Users/sakurai/Documents/learn-tensorflow/tensorflow/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
#データの準備

from keras.utils import np_utils
import numpy as np

categories = ["醤油ラーメン","味噌ラーメン","塩ラーメン","豚骨ラーメン","担々麺"]
nb_classes = len(categories)

X_train, X_test, y_train, y_test = np.load("./data/ramen_data.npy")

#データの正規化
X_train = X_train.astype("float") / 255
X_test  = X_test.astype("float")  / 255

#kerasで扱えるようにcategoriesをベクトルに変換
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test  = np_utils.to_categorical(y_test, nb_classes)

In [4]:
#モデルの学習

model = model.fit(X_train,
                  y_train,
                  epochs=10,
                  batch_size=6,
                  validation_data=(X_test,y_test))

W0819 15:44:42.605498 4373079488 deprecation_wrapper.py:119] From /Users/sakurai/Documents/learn-tensorflow/tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 381 samples, validate on 96 samples
Epoch 1/10
381/381 [==============================] - 20s 52ms/step - loss: 0.4892 - acc: 0.7974 - val_loss: 0.5147 - val_acc: 0.7604
Epoch 2/10
381/381 [==============================] - 20s 51ms/step - loss: 0.4222 - acc: 0.8157 - val_loss: 0.4329 - val_acc: 0.8125
Epoch 3/10
381/381 [==============================] - 19s 50ms/step - loss: 0.3783 - acc: 0.8325 - val_loss: 0.4419 - val_acc: 0.8083
Epoch 4/10
381/381 [==============================] - 20s 54ms/step - loss: 0.3479 - acc: 0.8483 - val_loss: 0.5334 - val_acc: 0.7625
Epoch 5/10
381/381 [==============================] - 19s 50ms/step - loss: 0.3073 - acc: 0.8772 - val_loss: 0.4186 - val_acc: 0.8396
Epoch 6/10
381/381 [==============================] - 19s 50ms/step - loss: 0.2786 - acc: 0.8882 - val_loss: 0.4291 - val_acc: 0.8187
Epoch 7/10
381/381 [==============================] - 19s 51ms/step - loss: 0.2389 - acc: 0.9097 - val_loss: 0.4996 - val_acc: 0.7875
Epoch 8/10
381/38

In [5]:
#学習結果を表示

import matplotlib.pyplot as plt

acc = model.history['acc']
val_acc = model.history['val_acc']
loss = model.history['loss']
val_loss = model.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('./data/accuracy.png')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('./data/loss.png')

In [6]:
#モデルの保存

json_string = model.model.to_json()
open('./data/ramen_predict.json', 'w').write(json_string)

#重みの保存

hdf5_file = "./data/ramen_predict.hdf5"
model.model.save_weights(hdf5_file)

In [7]:
from PIL import Image
import os, glob
import numpy as np
import random, math

# 画像が保存されているディレクトリのパス
root_dir = "./images"
# 画像が保存されているフォルダ名
categories = ["醤油ラーメン","味噌ラーメン","塩ラーメン","豚骨ラーメン","担々麺"]

X = [] # 画像データ
Y = [] # ラベルデータ

# フォルダごとに分けられたファイルを収集
#（categoriesのidxと、画像のファイルパスが紐づいたリストを生成）
allfiles = []
for idx, cat in enumerate(categories):
    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir + "/*.jpg")
    for f in files:
        allfiles.append((idx, f))

for cat, fname in allfiles:
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((256, 256))
    data = np.asarray(img)
    X.append(data)
    Y.append(cat)

x = np.array(X)
y = np.array(Y)

np.save("./data/ramen_data_test_X_256.npy", x)
np.save("./data/ramen_data_test_Y_256.npy", y)

In [8]:
# モデルの精度を測る

#評価用のデータの読み込み
test_X = np.load("./data/ramen_data_test_X_256.npy")
test_Y = np.load("./data/ramen_data_test_Y_256.npy")

#Yのデータをone-hotに変換
from keras.utils import np_utils

test_Y = np_utils.to_categorical(test_Y, 5) # 分類先の種類分設定

score = model.model.evaluate(x=test_X,y=test_Y)

print('loss=', score[0])
print('accuracy=', score[1])

477/477 [==============================] - 4s 8ms/step
loss= 1.7751038129474632
accuracy= 0.8876310198312035
